In [ ]:
import ngsolve as ngs
from ngsolve.webgui import Draw
import netgen.occ as occ

from dream.incompressible import IncompressibleSolver, flowfields, Inflow
import dream.bla as bla

shape = occ.Rectangle(20, 4.1).Circle(2, 2, 0.5).Reverse().Face()
shape.edges.name = "cyl"
shape.edges.Min(occ.X).name = "inlet"
shape.edges.Max(occ.X).name = "outlet"
shape.edges.Min(occ.Y).name = "wall"
shape.edges.Max(occ.Y).name = "wall"
mesh = ngs.Mesh(occ.OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.7))
Draw(mesh)

In [ ]:
# Set up the solver configuration
cfg = IncompressibleSolver(mesh)
cfg.fem = "taylor-hood"
# cfg.fem = "hdivhdg"
cfg.time = "stationary"

cfg.fem.order = 3
cfg.fem.scheme = "direct"
cfg.convection = False

cfg.reynolds_number = 100
cfg.dynamic_viscosity = "constant"

mesh.Curve(cfg.fem.order)

In [ ]:
cfg.bcs.clear()
cfg.dcs.clear()

# Set up the boundary conditions
cfg.bcs['inlet'] = Inflow(velocity=(1.5*4*ngs.y*(4.1-ngs.y)/(4.1*4.1), 0))  # flowfields(velocity=(1, 0)))
cfg.bcs['wall|cyl'] = "wall"
cfg.bcs['outlet'] = "outflow"

In [ ]:
# Initialize the finite element spaces, trial and test functions, gridfunctions, and boundary conditions
cfg.initialize()

Uh = cfg.get_solution_fields('velocity', default_fields=False)

cfg.io.draw(Uh)
with ngs.TaskManager():
    cfg.solve()
cfg.io.undraw()

In [ ]:
cfg.time = "transient"
cfg.fem.scheme = "imex"
cfg.time.timer.interval = (0, 100)
cfg.time.timer.step = 0.01
cfg.convection = True

cfg.fem.initialize_time_scheme_gridfunctions()
cfg.fem.initialize_symbolic_forms()

cfg.io.draw(Uh)
with ngs.TaskManager():
    cfg.solve()
cfg.io.undraw()
